# Feature Transformation with Amazon a SageMaker Processing Job and Scikit-Learn

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Scikit-Learn are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Scikit-Learn in a managed SageMaker environment to run our processing workload.

![](img/prepare_dataset_bert.png)

![](img/processing.jpg)


## Contents

1. Setup Environment
1. Setup Input Data
1. Setup Output Data
1. Build a Spark container for running the processing job
1. Run the Processing Job using Amazon SageMaker
1. Inspect the Processed Output Data

# Setup Environment

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [1]:
import sagemaker
from time import gmtime, strftime
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Setup Input Data

In [2]:
s3_raw_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_raw_input_data)

s3://sagemaker-us-west-2-393371431575/amazon-reviews-pds/tsv/


In [3]:
!aws s3 ls $s3_raw_input_data

2020-07-25 17:13:26   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-07-25 17:13:29   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz


# Run the Processing Job using Amazon SageMaker

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built, and a SparkML script for processing in the job configuration.

# Review the Processing Script

In [4]:
# !cat preprocess-scikit-text-to-bert.py
!pygmentize preprocess-scikit-text-to-bert.py

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import functools
import multiprocessing

import pandas as pd
from datetime import datetime
import subprocess
import sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
import tensorflow as tf
print(tf.__version__)
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
from transformers import DistilBertTokenizer
from tensorflow import keras
import os
import re
import collections
import argparse
import json
import os
import pandas as pd
import csv
import glob
from pathlib import Path

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

DATA_COLUMN = 'review_body'
LABEL_COLUMN = 'star_rating'
LABEL_VALUES = [1, 2, 3, 4, 5]
    
label_map = {}
for (i, label) in enumerate(LABEL_VALUES):
    label_map[label] = i

    
class InputFeatures(object):
  """BERT feature vectors."""

  def __init__(self,
               

Run this script as a processing job.  You also need to specify one `ProcessingInput` with the `source` argument of the Amazon S3 bucket and `destination` is where the script reads this data from `/opt/ml/processing/input` (inside the Docker container.)  All local paths inside the processing container must begin with `/opt/ml/processing/`.

Also give the `run()` method a `ProcessingOutput`, where the `source` is the path the script writes output data to.  For outputs, the `destination` defaults to an S3 bucket that the Amazon SageMaker Python SDK creates for you, following the format `s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name>/`.  You also give the `ProcessingOutput` value for `output_name`, to make it easier to retrieve these output artifacts after the job is run.

The arguments parameter in the `run()` method are command-line arguments in our `preprocess-scikit-text-to-bert.py` script.

Note that we sharding the data using `ShardedS3Key` to spread the transformations across all worker nodes in the cluster.

In [5]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

processor = SKLearnProcessor(framework_version='0.20.0',
                             role=role,
                             instance_type='ml.c5.2xlarge',
                             instance_count=2)

# Set the Train, Validation, Split Percentages

In [6]:
train_split_percentage = 0.90
validation_split_percentage = 0.05
test_split_percentage = 0.05

# Set the Maximum Sequence Length for the BERT Tokenizer

In [7]:
max_seq_length = 128

In [8]:
balance_dataset = False

In [9]:
processor.run(code='preprocess-scikit-text-to-bert.py',
              inputs=[ProcessingInput(source=s3_raw_input_data,
                                      destination='/opt/ml/processing/input/data/',
                                      s3_data_distribution_type='ShardedByS3Key')
              ],
              outputs=[
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='bert-train',
                                        source='/opt/ml/processing/output/bert/train'),
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='bert-validation',
                                        source='/opt/ml/processing/output/bert/validation'),
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='bert-test',
                                        source='/opt/ml/processing/output/bert/test'),
              ],
              arguments=['--train-split-percentage', str(train_split_percentage),
                         '--validation-split-percentage', str(validation_split_percentage),
                         '--test-split-percentage', str(test_split_percentage),
                         '--max-seq-length', str(max_seq_length),
                         '--balance-dataset', str(balance_dataset)
              ],
              logs=True,
              wait=False)

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  sagemaker-scikit-learn-2020-07-25-19-21-49-035
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-393371431575/amazon-reviews-pds/tsv/', 'LocalPath': '/opt/ml/processing/input/data/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-393371431575/sagemaker-scikit-learn-2020-07-25-19-21-49-035/input/code/preprocess-scikit-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'bert-train', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-393371431575/sagemaker-scikit-learn-2020-07-25-19-21-49-035/output/bert-train', 'LocalPath': '/opt/ml/processing/output/bert/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'bert-validation', 'S3Output': {'S3U

In [10]:
scikit_processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']
print(scikit_processing_job_name)

sagemaker-scikit-learn-2020-07-25-19-21-49-035


In [11]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, scikit_processing_job_name)))


In [12]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(bucket, scikit_processing_job_name, region)))


# List Processing Jobs through boto3 Python SDK

In [13]:
sm.list_processing_jobs()

{'ProcessingJobSummaries': [{'ProcessingJobName': 'sagemaker-scikit-learn-2020-07-25-19-21-49-035',
   'ProcessingJobArn': 'arn:aws:sagemaker:us-west-2:393371431575:processing-job/sagemaker-scikit-learn-2020-07-25-19-21-49-035',
   'CreationTime': datetime.datetime(2020, 7, 25, 19, 21, 49, 485000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2020, 7, 25, 19, 21, 49, 485000, tzinfo=tzlocal()),
   'ProcessingJobStatus': 'InProgress'},
  {'ProcessingJobName': 'sagemaker-scikit-learn-2020-07-25-19-18-17-480',
   'ProcessingJobArn': 'arn:aws:sagemaker:us-west-2:393371431575:processing-job/sagemaker-scikit-learn-2020-07-25-19-18-17-480',
   'CreationTime': datetime.datetime(2020, 7, 25, 19, 18, 17, 950000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2020, 7, 25, 19, 21, 29, 17000, tzinfo=tzlocal()),
   'ProcessingJobStatus': 'InProgress'},
  {'ProcessingJobName': 'spark-amazon-reviews-analyzer-2020-07-25-18-49-52-819',
   'ProcessingJobArn': 'arn:aws:sagemaker:

# Please Wait Until the Processing Job Completes
Re-run this next cell until the job status shows `Completed`.

In [14]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=scikit_processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

processing_job_status = processing_job_description['ProcessingJobStatus']
print('\n')
print(processing_job_status)
print('\n')

print(processing_job_description)



InProgress


{'ProcessingInputs': [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-393371431575/amazon-reviews-pds/tsv/', 'LocalPath': '/opt/ml/processing/input/data/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-393371431575/sagemaker-scikit-learn-2020-07-25-19-21-49-035/input/code/preprocess-scikit-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'bert-train', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-393371431575/sagemaker-scikit-learn-2020-07-25-19-21-49-035/output/bert-train', 'LocalPath': '/opt/ml/processing/output/bert/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'bert-validation', 'S3Output': {'S3Uri'

# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

In [15]:
running_processor.wait(logs=False)

.........................................................!

# Inspect the Processed Output Data

Take a look at a few rows of the transformed dataset to make sure the processing was successful.

In [16]:
output_config = processing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    if output['OutputName'] == 'bert-train':
        processed_train_data_s3_uri = output['S3Output']['S3Uri']
    if output['OutputName'] == 'bert-validation':
        processed_validation_data_s3_uri = output['S3Output']['S3Uri']
    if output['OutputName'] == 'bert-test':
        processed_test_data_s3_uri = output['S3Output']['S3Uri']
        
print(processed_train_data_s3_uri)
print(processed_validation_data_s3_uri)
print(processed_test_data_s3_uri)

s3://sagemaker-us-west-2-393371431575/sagemaker-scikit-learn-2020-07-25-19-21-49-035/output/bert-train
s3://sagemaker-us-west-2-393371431575/sagemaker-scikit-learn-2020-07-25-19-21-49-035/output/bert-validation
s3://sagemaker-us-west-2-393371431575/sagemaker-scikit-learn-2020-07-25-19-21-49-035/output/bert-test


In [17]:
!aws s3 ls $processed_train_data_s3_uri/

2020-07-25 19:26:30      50998 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-07-25 19:26:30      72302 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [18]:
!aws s3 ls $processed_validation_data_s3_uri/

2020-07-25 19:26:30       3224 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-07-25 19:26:30       4298 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [19]:
!aws s3 ls $processed_test_data_s3_uri/

2020-07-25 19:26:31       3255 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-07-25 19:26:31       4477 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


# Pass Variables to the Next Notebook(s)

In [20]:
%store s3_raw_input_data

Stored 's3_raw_input_data' (str)


In [21]:
%store max_seq_length

Stored 'max_seq_length' (int)


In [22]:
%store train_split_percentage

Stored 'train_split_percentage' (float)


In [23]:
%store validation_split_percentage

Stored 'validation_split_percentage' (float)


In [24]:
%store test_split_percentage

Stored 'test_split_percentage' (float)


In [25]:
%store balance_dataset

Stored 'balance_dataset' (bool)


In [26]:
%store processed_train_data_s3_uri

Stored 'processed_train_data_s3_uri' (str)


In [27]:
%store processed_validation_data_s3_uri

Stored 'processed_validation_data_s3_uri' (str)


In [28]:
%store processed_test_data_s3_uri

Stored 'processed_test_data_s3_uri' (str)


In [29]:
%store

Stored variables and their in-db values:
autopilot_endpoint_name                      -> 'automl-dm-ep-25-16-32-50'
balance_dataset                              -> False
comprehend_endpoint_arn                      -> 'arn:aws:comprehend:us-west-2:393371431575:documen
header_train_s3_uri                          -> 's3://sagemaker-us-west-2-393371431575/data/amazon
max_seq_length                               -> 128
noheader_train_s3_uri                        -> 's3://sagemaker-us-west-2-393371431575/data/amazon
processed_test_data_s3_uri                   -> 's3://sagemaker-us-west-2-393371431575/sagemaker-s
processed_train_data_s3_uri                  -> 's3://sagemaker-us-west-2-393371431575/sagemaker-s
processed_validation_data_s3_uri             -> 's3://sagemaker-us-west-2-393371431575/sagemaker-s
s3_raw_input_data                            -> 's3://sagemaker-us-west-2-393371431575/amazon-revi
test_split_percentage                        -> 0.05
train_split_percentage          

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();